In [1]:
import utils
from itertools import chain
from collections import Counter

In [2]:
raw = utils.load_raw("./data/信息抽取_第一阶段/xxcq_small.json")

## 文本长度统计

In [3]:
text_lengths = [len(line["text"]) for line in raw]

In [4]:
print(min(text_lengths), max(text_lengths))

7 439


## 标签统计

In [5]:
utils.LABEL_MEANING_MAP

{'NHCS': '犯罪嫌疑人',
 'NHVI': '受害人',
 'NCSM': '被盗货币',
 'NCGV': '物品价值',
 'NCSP': '盗窃获利',
 'NASI': '被盗物品',
 'NATS': '作案工具',
 'NT': '时间',
 'NS': '地点',
 'NO': '组织机构'}

In [6]:
entities = [line["entities"] for line in raw]
flatten_entities = list(chain(*entities))

In [7]:
entity_types = [entity[0] for entity in flatten_entities]

In [8]:
Counter(entity_types)

Counter({'NHCS': 2935,
         'NHVI': 1299,
         'NASI': 2555,
         'NT': 1226,
         'NS': 1580,
         'NCGV': 947,
         'NO': 346,
         'NATS': 294,
         'NCSP': 186,
         'NCSM': 418})

In [9]:
entities_text_lengths = [len(entity[-1]) for entity in flatten_entities]

In [12]:
sorted(Counter(entities_text_lengths).items(), key=lambda x: x[0])

[(1, 22),
 (2, 565),
 (3, 4337),
 (4, 764),
 (5, 725),
 (6, 476),
 (7, 595),
 (8, 651),
 (9, 504),
 (10, 415),
 (11, 397),
 (12, 428),
 (13, 311),
 (14, 311),
 (15, 241),
 (16, 173),
 (17, 162),
 (18, 120),
 (19, 99),
 (20, 95),
 (21, 70),
 (22, 67),
 (23, 70),
 (24, 37),
 (25, 28),
 (26, 29),
 (27, 21),
 (28, 21),
 (29, 10),
 (30, 5),
 (31, 10),
 (32, 6),
 (33, 4),
 (34, 4),
 (35, 4),
 (36, 1),
 (37, 1),
 (38, 1),
 (39, 1),
 (40, 2),
 (44, 1),
 (45, 1),
 (57, 1)]

In [10]:
print(min(entities_text_lengths), max(entities_text_lengths))

1 57
